# Extracting text with Grobid


Technically, this isn't an OpenReview snippet, but here's some code to grab text from pdfs, many of which you might have from OpenReview.

First, you have to set up Grobid. I'm mostly following [these instructions](https://grobid.readthedocs.io/en/latest/Install-Grobid/)
```
wget https://github.com/kermitt2/grobid/archive/0.7.1.zip
unzip 0.7.1.zip
rm 0.7.1.zip
```

You will need Gradle to run Grobid. I hope that the following works for you, because I don't know other ways to do this :-(
```
brew install gradle
cd grobid-0.7.1/
./gradlew clean install
cd ../

```

Create a directory in which you will save the pdfs to be extracted, e.g. `pdfs/`, and a directory where the xml files will be dumped, e.g. `xmls/`:

```
mkdir pdfs xmls
```

Copy your pdfs into this directory, then run:

```
java -Xmx4G -jar grobid-0.7.1/grobid-core/build/libs/grobid-core-0.7.1-onejar.jar \
	-gH grobid-0.7.1/grobid-home \
	-dIn pdfs/ \
	-dOut xmls/ \
	-exe processFullText 
```

Below is example code for extracting the section headers and text from a pdf.

In [ ]:
import glob

import xml.etree.ElementTree as ET

PREFIX = "{http://www.tei-c.org/ns/1.0}"

DIV_ID = f"{PREFIX}div"
HEAD_ID = f"{PREFIX}head"
P_ID = f"{PREFIX}p"

BIBLSTRUCT_ID = f"{PREFIX}biblStruct"
ANALYTIC_ID = f"{PREFIX}analytic"

def first_child(parent, child_part):
  return parent.findall(f'{PREFIX}{child_part}')[0]

def get_docs(filename):
  section_titles = []
  section_texts = []
  root = ET.parse(filename).getroot()
  divs = first_child(first_child(root, "text"), "body").findall(DIV_ID)
  for div in divs:
    header_node, = div.findall(HEAD_ID)
    section_titles.append(header_node.text)
    text = ""
    for p in div.findall(P_ID):
      text += " ".join(p.itertext())
    section_texts.append(text)
  return section_titles, section_texts

def get_references(filename):
  reference_titles = []
  
  divs = first_child(first_child(ET.parse(filename).getroot(), 'text'), 'back').findall(DIV_ID)
  for div in divs:
    if not div.attrib['type'] == 'references':
      continue
    for ref in first_child(div, "listBibl").findall(BIBLSTRUCT_ID):
      maybe_titles = ref.findall(ANALYTIC_ID)
      if maybe_titles:
        reference_titles.append(first_child(maybe_titles[0], 'title').text)
  return reference_titles

for filename in glob.glob("xmls/*"):
  sections, texts = get_docs(filename)
  for section, text in zip(sections, texts):
    print(section)
    print(text[:500] + "...")
    print()
  references = get_references(filename)
  for i in references:
    print(i)


# Extracting quotes with pdfalto

This requires you to install [pdfalto](https://github.com/kermitt2/pdfalto). This can be quite complicated. Also, I personally have not been able to get this to work. The fonts are somehow different between manuscripts at the same venue using the same template.

Anyway, if you have pdfalto installed in the ccurrent directory, do

```
cd pdfalto
./pdfalto /path/to/input.pdf temp.xml
```

then the code below will work (once you add the correct tests to select italics styles). I recommend using [an XML viewer](https://codebeautify.org/xmlviewer) to browse the xml file and figure out which styles apply to the content you want to extract.

In [ ]:
root =   ET.parse('pdfalto/temp.xml').getroot()

In [ ]:
PREFIX = '{http://www.loc.gov/standards/alto/ns-v3#}'
STYLES = f'{PREFIX}Styles'
TEXTSTYLE = f'{PREFIX}TextStyle'
LAYOUT = f'{PREFIX}Layout'
PAGE = f'{PREFIX}Page'
TEXT_BLOCK = f'{PREFIX}TextBlock'
STRING = f'{PREFIX}String'

In [ ]:
italics_styles = []
for style_child in root.findall(STYLES)[0].findall(TEXTSTYLE):
  # Uncomment to print all styles
  # print(style_child.attrib)
  if "i9" in style_child.attrib["FONTFAMILY"]: # You will need a different test for your document
    italics_styles.append(style_child.attrib["ID"])

In [ ]:
for page in root.findall(LAYOUT)[0].findall(PAGE):
  for print_space in page:
    for text_block in print_space.findall(TEXT_BLOCK):
      for text_line in text_block:
        flag = False
        for string in text_line.findall(STRING):
          if string.attrib['STYLEREFS'] in italics_styles or 'FONTSTYLE' in string.attrib and string.attrib['FONTSTYLE'] == 'italics':
            print(string.attrib["CONTENT"], end=" ")
            flag = True
        if flag:
          print("\n")